In [1]:
#encoding=utf8
import importlib

import urllib.request
from xlrd import open_workbook
from Bio import Entrez
import xml.dom.minidom
import xml.etree.ElementTree as ET
import pandas
import openpyxl

Entrez.email = "lhoang2@illinois.edu"

#Read list of PMIDs from txt file
#CHANGE THIS PATH TO THE FOLDER WHERE YOU STORE THE DATA
PMID_list_file = open("/Users/phuong/Downloads/NewData_PMIDs.txt","r")

new_PMIDs = []
for PMID in PMID_list_file.readlines():
    PMID = PMID.replace("\n","")
    new_PMIDs.append(PMID)
    
print (len(new_PMIDs))

#Save results to Excel file
from pandas import DataFrame
PMID_list = []
ArtcileTitle_list = []
Abstract_list = []
PublicationType_list = []
MeSH_list = []
PMCID_list = []

for PMID in new_PMIDs:
    print (PMID)
    #Pass PMIDs to PubMed API to get articles' metadata
    link = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + PMID + "&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y"            
    f = urllib.request.urlopen(link)   
    print (link)
    tree = ET.parse(f)
    doc = tree.getroot()    

    for child in doc:
        for step_child in child:
            if step_child.tag == "MedlineCitation":
                article_abstract_list = []
                article_publication_type_list = []
                article_MeSH_list = []
                
                article_abstract = "None"
                
                for step_child_level_2 in step_child:
                    if step_child_level_2.tag == "PMID":
                        article_PMID =  step_child_level_2.text
                      
                    if step_child_level_2.tag == "Article":
                        for step_child_level_3 in step_child_level_2:
                            if step_child_level_3.tag == "ArticleTitle":
                                article_title = step_child_level_3.text                         
                            
                            if step_child_level_3.tag == "Abstract":
                                if len(step_child_level_3) >= 1:                                    
                                    for step_child_level_4 in step_child_level_3:
                                        if step_child_level_4.tag == "AbstractText":
                                            article_abstract_list.append(step_child_level_4.text)
                                else:
                                    article_abstract_list.append("None")
                                article_abstract = b" ".join(str(x).encode('utf-8') for x in article_abstract_list)

                            
                            if step_child_level_3.tag == "PublicationTypeList":                                
                                for step_child_level_4 in step_child_level_3:
                                    if step_child_level_4.tag == "PublicationType":
                                        article_publication_type_list.append(step_child_level_4.text)
                                    else:
                                        article_publication_type_list.append("None")
                                                                             
                                article_publication_type = " ;".join(str(x) for x in article_publication_type_list)   
                                                                          
                        PMID_list.append(article_PMID)
                        ArtcileTitle_list.append(article_title)
                        Abstract_list.append(article_abstract)
                        PublicationType_list.append(article_publication_type)
                        
                    if step_child_level_2.tag == "MeshHeadingList":
                        for step_child_level_3 in step_child_level_2:
                            if step_child_level_3.tag == "MeshHeading":
                                for step_child_level_4 in step_child_level_3:
                                    if step_child_level_4.tag == "DescriptorName":
                                        article_MeSH_list.append(step_child_level_4.text)
                                
                                article_MeSH = " ;".join(str(x) for x in article_MeSH_list)  
                        MeSH_list.append(article_MeSH)
                        
                 
            if step_child.tag == "PubmedData":
                for step_child_level_2 in step_child:
                    if step_child_level_2.tag == "ArticleIdList":
                        for step_child_level_3 in step_child_level_2:
                            if (step_child_level_3.tag == "ArticleId"):
                                if (step_child_level_3.attrib['IdType'] == 'pmc'):
                                    PMCID = step_child_level_3.text
                                else:
                                    PMCID = "None"
                PMCID_list.append(PMCID)
                                
                        
print (len(PMID_list))
print (len(ArtcileTitle_list))
print (len(Abstract_list))
print (len(PublicationType_list))
print (len(PMCID_list))

df = DataFrame({'PMID': PMID_list , 'Article Title': ArtcileTitle_list, 'Abstract': Abstract_list, 
                'Publication Type': PublicationType_list,'PMCID':PMCID_list })
df = df[['PMID','Article Title', 'Abstract', 'Publication Type','PMCID']]

#CHANGE THIS PATH TO THE FOLDER WHERE YOU STORE THE DATA
df.to_excel('/Users/phuong/Downloads/NewData_Metadata.xlsx', sheet_name='sheet1', index=False)

156
23738582
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=23738582&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
12269976
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=12269976&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
29635061
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=29635061&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
24246616
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=24246616&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
26597253
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=26597253&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
19122337
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=19122337&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
24036846
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=24036846&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehisto

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=24962473&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
20078609
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=20078609&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
3221745
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=3221745&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
18492125
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=18492125&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
21171670
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=21171670&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
24100471
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=24100471&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
28155129
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=28155129&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
20590587
h

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=27628584&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
21496064
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=21496064&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
24036846
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=24036846&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
18503607
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=18503607&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
27518170
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=27518170&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
21171670
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=21171670&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
24962473
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=24962473&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
23738582